# Heart Disease Classification - Logistic Regression (LR)

## 1. Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## 2. Data Preprocessing

In [2]:
# Load dataframe (df)
df = pd.read_csv('heart.csv')
X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values

print(X.shape, y.shape)

(303, 13) (303,)


In [3]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [4]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [5]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [6]:
# Split dataframe (df) into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=0)

In [7]:
# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 3. Define LR Model

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [9]:
# Accuracy Scoring
lr_model = LogisticRegression(C=0.2, penalty='l2', class_weight='balanced', random_state=0)

# ROC_AUC Scoring
# lr_model = LogisticRegression(C=0.48, penalty='l2', class_weight='balanced', random_state=0)

# Train model
lr_model.fit(X_train, y_train)

LogisticRegression(C=0.2, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## 4. Evaluate LR Model

In [10]:
# Predict test set results
lr_pred = lr_model.predict(X_test)

# Create the confusion matrix
# TN = cm[0][0]  # True Negative
# FP = cm[0][1]  # False Positive
# FN = cm[1][0]  # False Negative
# TP = cm[1][1]  # True Positive
print('Confusion Matrix: ')
cm = confusion_matrix(y_test, lr_pred)
print(cm)
print('')
print("Test Accuracy: %.3f%% " % (accuracy_score(y_test, lr_pred) * 100))
print("ROC AUC: %.3f%% " % (roc_auc_score(y_test, lr_pred) * 100))
print('')

# accuracy = (TP + TN)/(TN + FP + FN + TP)
# precision = TP/(TP + FP)
# recall = TP/(TP + FN)
# f1_score = (2 * precision * recall)/(precision + recall)
print('Classification Report: ')
print(classification_report(y_test, lr_pred))

Confusion Matrix: 
[[21  6]
 [ 4 30]]

Test Accuracy: 83.607% 
ROC AUC: 83.007% 

Classification Report: 
              precision    recall  f1-score   support

           0       0.84      0.78      0.81        27
           1       0.83      0.88      0.86        34

    accuracy                           0.84        61
   macro avg       0.84      0.83      0.83        61
weighted avg       0.84      0.84      0.84        61



In [11]:
# Apply k-Fold Cross Validation on the training set

# cv = RepeatedKFold()
cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 0)
scores = cross_val_score(lr_model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# scores = cross_val_score(lr_model, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

# Force scores to be positive
scores = abs(scores)
print("Mean Accuracy: %.3f%%" % (np.mean(scores) * 100))
print("Standard Deviation: +/- %.3f%%" % np.std(scores))
print("Min Accuracy: %.3f%%" % (np.min(scores) * 100))
print("Max Accuracy: %.3f%%" % (np.max(scores) * 100))

Mean Accuracy: 82.489%
Standard Deviation: +/- 0.052%
Min Accuracy: 70.833%
Max Accuracy: 91.667%


<table border = "1">
  <thead>
    <th colspan = "3">Logistic Regression Cross-Validation Results</th>
  </thead>
  <tbody>
    <tr>
      <td><b></b></td>
      <td><b>Accuracy</b></td>
      <td><b>ROC_AUC</b></td>
    </tr>
    <tr>
      <td><b>Mean</b></td>
      <td>82.489%</td>
      <td>89.937%</td>
    </tr>
    <tr>
      <td><b>Std.</b></td>
      <td>0.052%</td>
      <td>0.045%</td>
    </tr>
    <tr>
      <td><b>Min</b></td>
      <td>70.833%</td>
      <td>79.720%</td>
    </tr>
    <tr>
      <td><b>Max</b></td>
      <td>91.667%</td>
      <td>97.203%</td>
    </tr>
  </tbody>
</table>

In [12]:
# Apply Grid Search for parameter tuning


# lr_params = {'C': [0.2, 0.25, 0.3, 0.5, 0.75, 1],
#               'penalty': ['l1', 'l2'],
#               'class_weight': ['dict', 'balanced'],
#               'random_state': [0, 1]}

# Accuracy scoring
lr_params = {'C': [0.19, 0.2, 0.21],
             'penalty': ['l1', 'l2'],
             'class_weight': ['dict', 'balanced'],
             'random_state': [0, 1]}

# ROC_AUC scoring
# lr_params = {'C': [0.46, 0.47, 0.48, 0.49],
#              'penalty': ['l1', 'l2'],
#              'class_weight': ['dict', 'balanced'],
#              'random_state': [0, 1]}

lr_grid = GridSearchCV(lr_model, lr_params, scoring='accuracy', cv=cv, n_jobs=-1)
# lr_grid = GridSearchCV(lr_model, lr_params, scoring='roc_auc', cv=cv, n_jobs=-1)
results = lr_grid.fit(X_train, y_train)

# Summarize results
best_score = lr_grid.best_score_
best_parameters = lr_grid.best_params_
print("Best Accuracy: %.3f%%" % (best_score * 100))
print("Best Parameters: ", best_parameters)


Best Accuracy: 82.489%
Best Parameters:  {'C': 0.2, 'class_weight': 'balanced', 'penalty': 'l2', 'random_state': 0}


<table border = "1">
  <thead>
    <th colspan = "3">Logistic Regression Grid Search Results</th>
  </thead>
  <tbody>
    <tr>
      <td><b></b></td>
      <td><b>Accuracy</b></td>
      <td><b>ROC_AUC</b></td>
    </tr>
    <tr>
      <td><b>Best Score</b></td>
      <td>82.489%</td>
      <td>89.937%</td>
    </tr>
    <tr>
      <td colspan="3"><b>Best Parameters:</b></td>
    </tr>
    <tr>
      <td><b>C</b></td>
      <td>0.2</td>
      <td>0.48</td>
    </tr>
    <tr>
      <td><b>class_weight</b></td>
      <td>balanced</td>
      <td>balanced</td>
    </tr>
    <tr>
      <td><b>penalty</b></td>
      <td>l2</td>
      <td>l2</td>
    </tr>
    <tr>
      <td><b>random_state</b></td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

## 5. Final Scores

<table border = "1">
  <thead>
    <th colspan = "3">LR Model Final Scores</th>
  </thead>
    <tr>
      <td><b></b></td>
      <td><b>Pred.</b></td>
      <td><b>Val.</b></td>
    </tr>
    <tr>
      <td><b>Accuracy (%):</b></td>
      <td>82.489</td>
      <td>83.607</td>
    </tr>
    <tr>
      <td><b>ROC AUC (%):</b></td>
      <td>89.937</td>
      <td>83.007</td>
    </tr>
  </tbody>
</table>